# Advanced Semantic Kernel - Advanced Agent Features

Welcome to the advanced agent features tutorial! This notebook builds on the basics to explore sophisticated individual agent capabilities and enhancements.

## Prerequisites

Before starting this tutorial, make sure you've completed the **Semantic Kernel Basics** notebook. You should be familiar with:
- Basic agent creation and configuration
- Chat completion services
- Plugins and function calling
- Conversation threads

## What You'll Learn

- **Streaming Responses**: Real-time response generation with `invoke_stream()`
- **Intermediate Steps**: Understanding agent processing with `invoke()`
- **Structured Outputs**: Enforcing consistent response formats with Pydantic
- **Reasoning Models**: Enhanced reasoning capabilities with specialized models

## Resources
- [Agent API Reference](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-api?pivots=programming-language-python)
- [Reasoning Models](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/reasoning?tabs=python-secure%2Cpy)

## Next Steps
After completing this tutorial, continue with the **Semantic Kernel Multi-Agent Orchestration** notebook to learn about agent collaboration patterns.

Let's explore these advanced agent features!

## Setup and Imports

Let's start by importing the necessary modules for advanced agent features.

In [ ]:
import asyncio
import json
import os
from dotenv import load_dotenv
from pydantic import BaseModel

try:
    from typing import Annotated
except ImportError:
    from typing_extensions import Annotated

# Core Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.agents import Agent, ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings, AzureChatPromptExecutionSettings
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.contents import ChatMessageContent, TextContent, StreamingChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent

# Load environment variables
load_dotenv()

print("✅ Advanced agent imports loaded successfully!")

✅ Advanced agent imports loaded successfully!


In [61]:
# Configure the main chat completion service
chat_completion = AzureChatCompletion(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
)

# Configure reasoning model (if available)
reasoning_completion = None
if os.getenv("AZURE_REASONING_ENDPOINT"):
    reasoning_completion = AzureChatCompletion(
        api_key=os.getenv("AZURE_REASONING_API_KEY"),
        endpoint=os.getenv("AZURE_REASONING_ENDPOINT"),
        deployment_name=os.getenv("AZURE_REASONING_DEPLOYMENT_NAME"),
        instruction_role="developer",
    )
    print("✅ Reasoning model available!")

print("✅ Chat completion services configured!")

✅ Reasoning model available!
✅ Chat completion services configured!


## 1. Advanced Agent Features

Let's explore advanced agent capabilities including streaming responses and intermediate step handling.

### 1.1 Streaming Responses

Streaming allows you to see responses as they're generated, providing a better user experience for long responses.

In [62]:
# Create a simple agent for streaming demonstration
streaming_agent = ChatCompletionAgent(
    service=chat_completion,
    name="StreamingExpert",
    instructions="You are an expert who provides detailed explanations. Give comprehensive responses about topics.",
)

In [63]:
# Example 1: Regular (non-streaming) response
print("📝 Regular Response:")
print("=" * 40)
response = await streaming_agent.get_response(
    messages="Explain the benefits of renewable energy in detail."
)
print(response.message.content)

print("\n" + "=" * 40)

📝 Regular Response:
Renewable energy—power drawn from naturally replenishing sources such as sunlight, wind, water, biomass and geothermal heat—offers a host of long-term advantages over fossil fuels. Below is a detailed look at the principal benefits:

1. Environmental Benefits  
   • Greenhouse Gas Reductions  
     – Solar panels, wind turbines and run-of-river hydro generate electricity without burning carbon-based fuels.  
     – Widespread adoption can slash CO₂ emissions, a key driver of global warming and climate change.  
   • Improved Air Quality  
     – Fossil-fuel power plants emit particulate matter (PM₂.₅ and PM₁₀), sulfur dioxide (SO₂) and nitrogen oxides (NOₓ). Renewables produce little to no air pollutants, reducing smog and respiratory illnesses.  
   • Decreased Water Use and Pollution  
     – Thermal plants (coal, natural gas, nuclear) require vast amounts of water for cooling. Most renewables (wind, solar PV) need no water; hydro and geothermal use water without 

In [64]:
# Example 2: Streaming response using invoke_stream()
print("🌊 Streaming Response:")
print("=" * 40)
async for streaming_response in streaming_agent.invoke_stream(
    messages="Explain the challenges of implementing AI in healthcare."
):
    # This shows the response as it's being generated in real-time
    print(streaming_response.content, end="", flush=True)

print("\n\n✅ Streaming examples completed!")

# Create an agent with the analysis plugin for the next section
analysis_agent = ChatCompletionAgent(
    service=chat_completion,
    name="AnalysisExpert",
    instructions="You are an expert analyst. Use your tools to provide thorough analysis and recommendations.",
    plugins=[AnalysisPlugin()]
)

print("✅ Analysis agent created for next section!")

🌊 Streaming Response:
Implementing AI in healthcare promises improved diagnostics, personalized treatment and operational efficiencies—but it also comes with a host of challenges. Key hurdles include:

1. Data-Related Challenges  
   • Quality and consistency: Clinical data often reside in disparate systems and formats (free text notes, scans, lab values), making standardization and cleaning difficult.  
   • Volume and labeling: Many AI models require large, well-annotated datasets. Labeling medical images or records is time-consuming and typically requires expert clinicians.  
   • Interoperability: Electronic health records (EHRs) from different vendors rarely “speak the same language,” hampering data exchange and aggregation for AI training.

2. Privacy, Security and Compliance  
   • Patient confidentiality: Regulations such as HIPAA (U.S.) or GDPR (EU) impose strict controls on how patient data can be used, shared or de-identified.  
   • Cybersecurity risks: Centralizing sensiti

### 1.2 Intermediate Steps with invoke()

**Understanding Agent Methods:**

- **`get_response()`**: Provides the final response only - blocks until the agent is completely done processing
- **`invoke()`**: Allows you to see the agent's step-by-step thinking process, including intermediate function calls and results
- **`invoke_stream()`**: Provides real-time streaming of the response as it's being generated

The `invoke()` method is particularly useful when you want to understand how the agent is working with plugins and function calls.

In [65]:
class AnalysisPlugin:
    """Plugin that provides analysis capabilities."""
    
    @kernel_function(description="Analyze a topic and provide insights.")
    def analyze_topic(self, topic: Annotated[str, "The topic to analyze"]) -> str:
        # Simulate analysis processing
        return f"Analysis of '{topic}': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility."
    
    @kernel_function(description="Generate recommendations based on analysis.")
    def generate_recommendations(self, analysis: Annotated[str, "The analysis to base recommendations on"]) -> str:
        return "Recommendations: 1) Conduct further research, 2) Develop a prototype, 3) Test with users, 4) Iterate based on feedback."


In [66]:
async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    """Handle intermediate steps in the agent's processing."""
    for item in message.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"🔧 Function Result: {item.result}")
        elif isinstance(item, FunctionCallContent):
            print(f"📞 Function Call: {item.name} with arguments: {item.arguments}")
        else:
            print(f"💭 {message.name}: {message.content}")

# Test intermediate steps
task = "Analyze the potential of AI-powered educational tools and provide recommendations."
print(f"🎯 Task: {task}")
print("=" * 60)

async for response in analysis_agent.invoke(
    messages=task,
    on_intermediate_message=handle_intermediate_steps
):
    print(f"✅ Final Response from {response.name}: {response.content}")

🎯 Task: Analyze the potential of AI-powered educational tools and provide recommendations.
📞 Function Call: AnalysisPlugin-analyze_topic with arguments: {"topic":"AI-powered educational tools"}
🔧 Function Result: Analysis of 'AI-powered educational tools': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility.
📞 Function Call: AnalysisPlugin-generate_recommendations with arguments: {"analysis":"1. Market Trends: The global edtech market is projected to expand at a CAGR of over 15% through the next five years, driven by increased digital adoption, remote learning needs, and demand for personalized instruction.\n\n2. Learner Needs: Students across age groups seek adaptive, self-paced learning with instant feedback. Educators need tools that reduce administrative burden, support differentiated instruction, and align with curriculum standards.\n\n3. Technical Feasibility: Advances in natural language p

In [67]:
# Straming with intermediate steps
async for streaming_response in analysis_agent.invoke_stream(
    messages=task,
    on_intermediate_message=handle_intermediate_steps
):
    print(streaming_response.content, end="", flush=True)
    

📞 Function Call: AnalysisPlugin-analyze_topic with arguments: {"topic":"AI-powered educational tools"}
🔧 Function Result: Analysis of 'AI-powered educational tools': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility.
📞 Function Call: AnalysisPlugin-analyze_topic with arguments: {"topic":"AI-powered educational tools"}
🔧 Function Result: Analysis of 'AI-powered educational tools': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility.
📞 Function Call: AnalysisPlugin-generate_recommendations with arguments: {"analysis":"AI-powered educational tools are showing significant growth potential due to several factors:\n• Market Trends: The global edtech market is expanding rapidly, projected to reach $404B by 2025. AI adoption within edtech is accelerating, fueled by demand for personalized learning, analytics, and

### 1.3 Structured Outputs with Pydantic

You can enforce structured responses using Pydantic models, ensuring consistent output format.

In [68]:
# Define structured output models
class Step(BaseModel):
    explanation: str
    output: str

class MathSolution(BaseModel):
    steps: list[Step]
    final_answer: str

# Create an agent with structured output
settings = AzureChatPromptExecutionSettings()
settings.response_format = MathSolution

math_agent = ChatCompletionAgent(
    service=chat_completion,
    name="MathTutor",
    instructions="You are a math tutor. Solve problems step by step and provide clear explanations.",
    arguments=KernelArguments(settings=settings)
)

# Test structured output
problem = "Solve: 3x + 7 = 22"
print(f"🧮 Problem: {problem}")

response = await math_agent.get_response(messages=problem)
solution = MathSolution.model_validate(json.loads(response.message.content))

print("\n📚 Structured Solution:")
print(solution.model_dump_json(indent=2))

🧮 Problem: Solve: 3x + 7 = 22

📚 Structured Solution:
{
  "steps": [
    {
      "explanation": "Subtract 7 from both sides to isolate the term with x.",
      "output": "3x = 22 - 7 = 15"
    },
    {
      "explanation": "Divide both sides by 3 to solve for x.",
      "output": "x = 15 / 3 = 5"
    }
  ],
  "final_answer": "x = 5"
}


### 1.4 Reasoning Models (if available)

Some models like O1 provide enhanced reasoning capabilities. Let's test if we have access to reasoning models.

In [69]:
if reasoning_completion:
    print("🧠 Testing reasoning model...")
    
    reasoning_settings = AzureChatPromptExecutionSettings(
        service_id="reasoning",
        reasoning_effort="high"
    )
    
    reasoning_agent = ChatCompletionAgent(
        name="ReasoningExpert",
        instructions="You are an expert problem solver. Think through complex problems step by step.",
        service=reasoning_completion,
    )
    
    thread = ChatHistoryAgentThread()
    response = await reasoning_agent.get_response(
        messages="Solve: 2x + 3y = 7, 4x - y = 1. Show your reasoning.", 
        thread=thread
    )
    
    print(f"🧠 Reasoning Response: {response.message.content}")

    # Add another user message to the thread
    print("\n🤔 User: Is the solution correct? Explain.")
    response = await reasoning_agent.get_response(
        messages="Is the solution correct? Explain.", 
        thread=thread
    )
    print(f"🧠 Reasoning Response: {response.message.content}")
else:
    print("⚠️ Reasoning model not configured. Skipping reasoning example.")
    print("💡 To enable reasoning, configure AZURE_REASONING_* environment variables.")

🧠 Testing reasoning model...
🧠 Reasoning Response: Here’s a step‐by‐step solution by elimination:

We want to solve  
(1) 2x + 3y = 7  
(2) 4x − y = 1  

1. Eliminate y.  Multiply (2) by 3:  
   3·(4x − y) = 3·1  
   ⇒ 12x − 3y = 3  

2. Add this to (1):  
   (2x + 3y) + (12x − 3y) = 7 + 3  
   ⇒ 14x = 10  
   ⇒ x = 10/14 = 5/7  

3. Substitute x = 5/7 back into, say, (1):  
   2·(5/7) + 3y = 7  
   ⇒ 10/7 + 3y = 7  
   ⇒ 3y = 7 − 10/7 = 49/7 − 10/7 = 39/7  
   ⇒ y = (39/7)/3 = 39/21 = 13/7  

4. (Optional check in (2)):  
   4·(5/7) − (13/7) = 20/7 − 13/7 = 7/7 = 1 ✓  

Answer:  
x = 5/7, y = 13/7.

🤔 User: Is the solution correct? Explain.
🧠 Reasoning Response: Yes. You can verify by substituting x=5/7 and y=13/7 into both equations:

1. 2x + 3y  
   = 2·(5/7) + 3·(13/7)  
   = 10/7 + 39/7  
   = 49/7  
   = 7  ✓

2. 4x − y  
   = 4·(5/7) − (13/7)  
   = 20/7 − 13/7  
   = 7/7  
   = 1  ✓

Since both original equations are satisfied, the solution x=5/7, y=13/7 is indeed correct.


In [70]:
# You can see the reasoning tokens used by the agent
async for message in thread.get_messages():
    print(f"📝 Message from {message.role}: {message.content[:20]}...")
    if message.metadata.get("usage", None):
        print(f"    🔍 Usage: {message.metadata['usage'].model_dump()}")

📝 Message from AuthorRole.USER: Solve: 2x + 3y = 7, ...
📝 Message from AuthorRole.ASSISTANT: Here’s a step‐by‐ste...
    🔍 Usage: {'prompt_tokens': 55, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'completion_tokens': 902, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}}
📝 Message from AuthorRole.USER: Is the solution corr...
📝 Message from AuthorRole.ASSISTANT: Yes. You can verify ...
    🔍 Usage: {'prompt_tokens': 383, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'completion_tokens': 429, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}}


In [71]:
await thread.delete()

## 🎉 Congratulations!

You've successfully completed the Advanced Agent Features tutorial! Here's what you've mastered:

### ✅ Advanced Agent Capabilities:
1. **Streaming Responses** - Real-time response generation with `invoke_stream()`
2. **Intermediate Steps** - Understanding agent processing with `invoke()` and callbacks
3. **Structured Outputs** - Enforcing consistent response formats with Pydantic models
4. **Reasoning Models** - Enhanced reasoning capabilities with specialized models and token tracking

### 🔧 Key Agent Methods:
- **`get_response()`** - Blocking call that returns the final response
- **`invoke()`** - Non-blocking call that shows intermediate steps and function calls
- **`invoke_stream()`** - Real-time streaming of responses as they're generated

### 🚀 What's Next?
Now that you've mastered individual agent capabilities, you're ready to explore multi-agent systems! Continue with the **Semantic Kernel Multi-Agent Orchestration** notebook to learn about:
- Group chat orchestration patterns
- Concurrent and sequential processing
- Handoff systems
- Human-in-the-loop workflows

### 📚 Additional Resources:
- [Agent API Reference](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-api?pivots=programming-language-python)
- [Reasoning Models Documentation](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/reasoning?tabs=python-secure%2Cpy)
- [Plugin Development](https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/)

Great work! You're now ready to build sophisticated AI agents with advanced capabilities. 🚀